<a href="https://colab.research.google.com/github/MexJason/NeuralNetworkPlayground/blob/main/neural-network-from-scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

In [4]:
num_layers = 3
layer_size = [12, 8, 4, 1]
input_size = 12

W1 = np.random.randn(layer_size[1], input_size)* 0.01
b1 = np.zeros((layer_size[1],1))
W2 = np.random.randn(layer_size[2], layer_size[1])* 0.01
b2 = np.zeros((layer_size[2],1))
W3 = np.random.randn(layer_size[3], layer_size[2])* 0.01
b3 = np.zeros((layer_size[3],1))

params = [W1, b1, W2, b2, W3, b3]
W1, W2, W3

(array([[-7.60455520e-03, -1.65839048e-02, -9.36033798e-03,
          3.31282554e-04,  8.32794740e-03,  2.12257625e-02,
          6.40499753e-03,  1.40248664e-02, -1.39409386e-02,
         -1.01519045e-02, -2.43724231e-03,  7.24446823e-03],
        [-1.19410944e-03, -9.10544091e-03, -1.38655811e-02,
          3.12102856e-03,  1.78979356e-02,  7.21372790e-04,
          1.05815797e-02, -8.87024815e-03, -2.83426700e-03,
          1.09582872e-02,  1.53741555e-02,  5.64490349e-03],
        [ 1.66184138e-02, -7.65801911e-03,  1.07363349e-02,
         -4.20968125e-03,  8.31167623e-04, -9.13677570e-03,
          5.05999374e-03,  1.92484641e-03, -9.63352655e-03,
         -9.23306217e-03, -6.88818268e-03, -2.96735458e-03],
        [-7.76746790e-04,  3.08099156e-02,  9.82272589e-03,
          7.79891441e-03,  3.16508248e-03,  5.98805620e-03,
         -6.21404727e-03,  2.82495936e-03,  6.91802912e-03,
         -4.37977102e-04, -1.50613825e-02, -5.72443556e-03],
        [-6.62063805e-03, -7.6476005

In [5]:
# Number of paramaters in model 
num_params = sum(p.size for p in params)
num_params

145

In [6]:
layer_dims = [12, 8, 4, 1]

parameters = {}
L = len(layer_dims) # number of layers in the network

for l in range(1, L):
    parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    
#num_params = sum(p.size for p in parameters)
num_params = 0
for p in parameters:
  print(p, parameters[p].shape)
  num_params += parameters[p].size
print(num_params)

W1 (8, 12)
b1 (8, 1)
W2 (4, 8)
b2 (4, 1)
W3 (1, 4)
b3 (1, 1)
145


In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [8]:
def linear_forward(A, W, b):
    Z = W @ A + b
    cache = (A, W, b)
      
    return Z, cache

In [9]:
def linear_activation_forward(A_prev, W, b, activation):
  if activation == "sigmoid":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A = sigmoid(Z)
    activation_cache = A * (1 - A)
  elif activation == "relu":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A = np.maximum(0, Z)
    activation_cache = A * (1 - A)

  cache = (linear_cache, activation_cache)

  return A, cache

In [41]:
def L_model_forward(X, parameters):
  """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- activation value from the output (last) layer
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """
  caches = []
  A = X
  L = len(parameters) // 2           

  for l in range(1, L):
      A_prev = A 
      A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)],'relu')
      caches.append(cache)
  # MAKE SURE TO USE CURRENT A!!!
  AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)],'sigmoid')
  caches.append(cache)


  print(AL.shape, len(caches))
  return AL, caches

In [42]:
# calculations we will need in backward pass 
def linear_backward(dZ, cache):
  A_prev, W, b = cache
  m = A_prev.shape[1]
  
  dW = dZ @ A_prev.T * (1/m)
  db = np.sum(dZ, axis=1, keepdims=True)* (1/m)
  dA_prev = W.T @ dZ

  return dA_prev, dW, db

In [43]:
def relu_backward(dA, activation_cache):
  Z = activation_cache
  dZ = np.array(dA, copy=True)
  dZ[Z <= 0] = 0
    
  return dZ

def sigmoid_backward(dA, activation_cache):
  Z = activation_cache
  s = 1/(1+np.exp(-Z))
  dZ = dA * s * (1-s)
    
  return dZ

In [44]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
      dZ = relu_backward(dA, activation_cache)
      dA_prev, dW, db = linear_backward(dZ, linear_cache)       

    elif activation == "sigmoid":
      dZ = sigmoid_backward(dA, activation_cache)
      dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [45]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, "sigmoid")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [61]:
def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    #(≈ 2 lines of code)
    for l in range(L):
        # parameters["W" + str(l+1)] = ...
        # parameters["b" + str(l+1)] = ...
        # YOUR CODE STARTS HERE
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [62]:
layer_dims = [12, 8, 4]
X = np.random.randn(layer_dims[0], layer_dims[1])
Y = np.random.randn(layer_dims[-1], layer_dims[-2])
print(Y.shape)
learning_rate = 0.1

parameters = {}
L = len(layer_dims) # number of layers in the network

for l in range(1, L):
    parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

for i in range(50):

  # Forward Pass
  AL, caches = L_model_forward(X, parameters)

  # Backward Pass
  grads = L_model_backward(AL, Y, caches)

  # Update
  params = update_parameters(parameters, grads, learning_rate)

print("W1 = "+ str(params["W1"]))
print("b1 = "+ str(params["b1"]))
print("W2 = "+ str(params["W2"]))
print("b2 = "+ str(params["b2"]))

(4, 8)
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 8)
1 (8, 8)
(4, 8) 2
(12, 